# Present the symbolic derivation of existing models

In [ ]:
%matplotlib widget
import os, importlib, inspect
from IPython.display import display, Math, Latex, Markdown
from bmcs_matmod.gsm.gsm_models.gsm_base import GSMBase, sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
modules = []

# Collect all relevant modules and their classes
for file_name in os.listdir('.'):
    if file_name.endswith('.py') and file_name not in ['gsm_base.py', '__init__.py']:
        module_path = f"bmcs_matmod.gsm.gsm_models.{file_name[:-3]}"
        module = importlib.import_module(module_path)
        for name, obj in inspect.getmembers(module, inspect.isclass):
            if issubclass(obj, GSMBase) and obj is not GSMBase:
                modules.append((name, obj))

# Sort the modules by class names
modules.sort(key=lambda x: x[0])

gsm_models = {name: obj() for name, obj in modules}
gsm_models

In [ ]:
n_t = 151
n_I = 1
eps_max = 5
t_t = np.linspace(0, 10, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
response = {}
for name, gsm in gsm_models.items():
    print(f'name: {name}')
    gsm_engine = gsm.F_engine
    mparams = {var: 1 for var in gsm_engine.m_params}
    response[name] = gsm_engine.get_response(eps_n_t, t_t, mparams, 10)


In [ ]:
fig, ax_sig = plt.subplots(1, 1, figsize=(8, 5))

selection = ['GSM1D_VEVP', 'GSM1D_VEVPD', 'GSM1D_VED']
if len(selection) == 0:
    selection = response.keys()
for name in selection:
    resp = response[name]    
    t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = resp
    eps_t = eps_ta[:, 0]
    sig_t = sig_ta[:, 0, 0]
    ax_sig.plot(eps_t, sig_t, label=rf'{name}')

ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()


In [ ]:
E, K, f_c, eta_vp, eta_ve = gsm_models['GSM1D_VEVP'].F_engine.m_params # S, c, r, 
#mparams = {E: 100, K: 10, f_c: 1, S: 1, c: 1, r: 1, eta_vp: 1, eta_ve: 1}
mparams = {E: 100, K: 10, f_c: 1, eta_vp: 1, eta_ve: 1}
response = gsm_models['GSM1D_VEVP'].F_engine.get_response(eps_n_t, t_t, mparams, 10)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
eps_t = eps_ta[:, 0]
sig_t = sig_ta[:, 0, 0]
# eps_ve_t, eps_vp_t, omega_t, z_t = Eps_t[:, 0, :].T
# sig_ve_t, sig_vp_t, Y_t, Z_t = Sig_t[:, 0, :].T
eps_ve_t, eps_vp_t, z_t = Eps_t[:, 0, :].T
sig_ve_t, sig_vp_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, ax_sig = plt.subplots(1, 1, figsize=(8, 5))
ax_sig.plot(eps_t, sig_t, 'o-', label=rf'{name} - modified')

In [ ]:
fig, (ax_sig, ax_omega) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t - eps_vp_t, sig_vp_t, ls='dashed', color='red', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.plot(eps_t - eps_ve_t, sig_ve_t, ls='dashed', color='orange', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.plot(eps_t - eps_ve_t, sig_ve_t, ls='dashed', color='orange', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray')
ax_iter.set_ylabel(r'$k$', color='gray')

# ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='black', lw=1)
ax_eps.plot(t_t, eps_t - eps_ve_t - eps_vp_t, label=r'$\varepsilon_\mathrm{e}$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_ve_t, label=r'$\varepsilon_\mathrm{ve}$', color='red', ls='dashed', lw=1)
ax_eps.plot(t_t, eps_vp_t, label=r'$\varepsilon_\mathrm{vp}$', color='orange', ls='dashed', lw=1)
#ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')
ax_eps.legend()

plt.tight_layout()
plt.show()